In [355]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import scipy


In [560]:

# class RBM(nn.Module):
class RBM():
    def __init__(self,
                 n_vis=4499,
                 n_hin=100,
                 k=5,
                 use_gpu=False
                ):
#         super(RBM, self).__init__()
        self.desc = "RBM"
        
#         self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
#         self.v_bias = nn.Parameter(torch.zeros(1, n_vis))
#         self.h_bias = nn.Parameter(torch.zeros(1 ,n_hin))

        self.W = torch.randn(n_hin,n_vis)*1e-2
        self.v_bias = torch.rand(1, n_vis)
        self.h_bias = torch.rand(1 ,n_hin)
        self.k = k
        #self.myfunc = F.logsigmoid
    
        
        self.use_gpu = use_gpu
        if self.use_gpu:
            self.W = self.W.cuda()
            self.h_bias = self.h_bias.cuda()
            self.v_bias = self.v_bias.cuda()
    
    def v_to_h(self,v):
        
        p_h =None
        sample_h = None
#         try:
        #计算得到：1.s型激活函数的x值：w*v+hbia;2.隐藏层神经元为1的概率：P(h=1|v)
        v_activation = F.linear(v,self.W,self.h_bias)
        
#         wv = torch.mm(v, self.W.t())
#         v_activation = wv + self.a.expand_as(wv)
        
        p_h = F.softmax(v_activation,dim=1)
        #采样隐藏层h的值
        sample_h = torch.bernoulli(p_h)
#         except:
#             print("v",v)
#             print("w",self.W)
#             print("h",self.h_bias)
        return p_h, sample_h
    
    def h_to_v(self,h):
        h_activation = F.linear(h,self.W.t(),self.v_bias)
#         wh = torch.mm(h, self.W)
#         h_activation = torch.mm(h, self.W) + self.b.expand_as(wh)
        
        p_v = F.sigmoid(h_activation)
        sample_v = torch.bernoulli(p_v)
        return p_v, sample_v
        
    def forward(self,v):
        # do contrastive ydivergence
        # Positive phase
        pre_h1,_ = self.v_to_h(v)
        # Negative phase; gibbs采样 
        for k in range(self.k):
            _,hk = self.v_to_h(v)
            _,vk = self.h_to_v(hk)
            # mask
            vk[v <0] = v[v<0]
        pre_hk,_ = self.v_to_h(vk)
        return vk,pre_h1,pre_hk
    
    def free_energy(self,v):
        wx_b = F.linear(v, self.W, self.h_bias)
        wx_b = (wx_b - wx_b.min()) / (wx_b.max() - wx_b.min())
        # sum all hidden node :ln(1+exp(w*v+b))
        hidden_term = wx_b.exp().add(1).log().sum(1)
        #hidden_term = F.logsigmoid(data).add(1).sum(1)
        vbias_term = v.mm(self.v_bias.t())
        return (-hidden_term - vbias_term).mean()
    
#     def test(self,v):
#         pre_h1,h1 = self.v_to_h(v)
#         h_ = h1
#         pre_v_,v_ = self.h_to_v(h_)
#         return v_
    
    def train(self, v, vk, ph, phk):
        # positive associations - negative associations
        a = torch.matmul(v.t(), ph) - torch.matmul(vk.t(), phk)
        self.W += a.t()
        # input_data - negative_visible_probabilities
        self.v_bias += torch.sum((v - vk), 0)
        # positive_hidden_probabilities - negative_hidden_probabilities
        self.h_bias += torch.sum((ph - phk), 0)


In [561]:
# npdata = np.loadtxt("./dataset/np_matrix_1.txt") 
# print("shape: ",npdata.shape)

In [563]:
batch_size = 10
k = 1
lr = 0.0003
visible_num = npdata.shape[1]
hidden_num = 10

rbm = RBM(k=k,n_vis = visible_num, n_hin=hidden_num)
# train_op = optim.SGD(rbm.parameters(),lr)

In [564]:
npdata.shape

(2511, 4499)

In [573]:
for epoch in range(10):
    loss_ = []
    for i in range(0, npdata.shape[0] - batch_size-1, batch_size):
        print("i = ",i)
        data = Variable(torch.from_numpy(npdata[i:i+batch_size]).type('torch.FloatTensor'))
        vk,pre_h1,pre_hk = rbm.forward(data)
        
#         # loss = 所有用户的自由能F(v)的均值的差; 
        loss = rbm.free_energy(data)- rbm.free_energy(vk)
#         if(loss<0):
#             print("loss",loss)
# #             print(data[data>0])
#             a = vk.t().max(1, keepdim=True)[0]
#             print(a[a>1])
            
#             break
        
#         loss_.append(loss.data[0])
#         train_op.zero_grad()
#         loss.backward()
#         train_op.step()
        rbm.train(data, vk, pre_h1, pre_hk)
    
        loss_.append(torch.mean(torch.abs(data[data>=0] - vk[data>=0])))

        print(np.mean(loss_))

i =  0
0.06365859
i =  10
0.067303844
i =  20
0.06421427
i =  30
0.06393643
i =  40
0.062316068
i =  50
0.05844261
i =  60
0.059549738
i =  70
0.06639531
i =  80
0.06869181
i =  90
0.068370745
i =  100
0.06898704
i =  110
0.069500625
i =  120
0.069769345
i =  130
0.07118724
i =  140
0.0716989
i =  150
0.07272866
i =  160
0.07375234
i =  170
0.074623995
i =  180
0.07503305
i =  190
0.075749055
i =  200
0.0760423
i =  210
0.07507426
i =  220
0.0775815
i =  230
0.07765985
i =  240
0.079037115
i =  250
0.07910903
i =  260
0.07825278
i =  270
0.07702426
i =  280
0.076539606
i =  290
0.07594428
i =  300
0.07698413
i =  310
0.07744291
i =  320
0.077100635
i =  330
0.07652093
i =  340
0.076396026
i =  350
0.07613605
i =  360
0.07559938
i =  370
0.07566009
i =  380
0.07546634
i =  390
0.076650366
i =  400
0.07740854
i =  410
0.077258974
i =  420
0.07697887
i =  430
0.07682414
i =  440
0.07641946
i =  450
0.07636045
i =  460
0.0761408
i =  470
0.07559643
i =  480
0.07539453
i =  490
0.07538698
i

0.07471373
i =  1510
0.0746685
i =  1520
0.07462108
i =  1530
0.07458136
i =  1540
0.07471782
i =  1550
0.0748427
i =  1560
0.07487
i =  1570
0.074817464
i =  1580
0.07467949
i =  1590
0.0745274
i =  1600
0.07443531
i =  1610
0.07438114
i =  1620
0.074334316
i =  1630
0.07421541
i =  1640
0.07417581
i =  1650
0.07413214
i =  1660
0.07409498
i =  1670
0.074066065
i =  1680
0.07394635
i =  1690
0.07381536
i =  1700
0.07366173
i =  1710
0.07354943
i =  1720
0.07350086
i =  1730
0.07361381
i =  1740
0.07369613
i =  1750
0.07368734
i =  1760
0.07369221
i =  1770
0.07367293
i =  1780
0.07370105
i =  1790
0.073824674
i =  1800
0.074014485
i =  1810
0.07411024
i =  1820
0.07403564
i =  1830
0.0739238
i =  1840
0.073872395
i =  1850
0.07382262
i =  1860
0.07368815
i =  1870
0.07356729
i =  1880
0.073465474
i =  1890
0.073416196
i =  1900
0.073424816
i =  1910
0.07354343
i =  1920
0.07366542
i =  1930
0.073685676
i =  1940
0.073591396
i =  1950
0.07350602
i =  1960
0.073502295
i =  1970
0.073575

0.07727631
i =  590
0.077610955
i =  600
0.07749117
i =  610
0.07799224
i =  620
0.07824314
i =  630
0.077989556
i =  640
0.078084365
i =  650
0.07822345
i =  660
0.07860321
i =  670
0.07867305
i =  680
0.07852277
i =  690
0.07842696
i =  700
0.07853451
i =  710
0.078573324
i =  720
0.07853374
i =  730
0.07825824
i =  740
0.07818301
i =  750
0.07812731
i =  760
0.0780413
i =  770
0.07829746
i =  780
0.07838283
i =  790
0.078276284
i =  800
0.0784531
i =  810
0.07842935
i =  820
0.078217916
i =  830
0.07800913
i =  840
0.07771792
i =  850
0.07739006
i =  860
0.077210255
i =  870
0.07697595
i =  880
0.07692471
i =  890
0.07679682
i =  900
0.07651909
i =  910
0.076259464
i =  920
0.07601116
i =  930
0.07600909
i =  940
0.07615025
i =  950
0.076065056
i =  960
0.07625979
i =  970
0.07626502
i =  980
0.07605327
i =  990
0.07598378
i =  1000
0.07605893
i =  1010
0.07649304
i =  1020
0.0768406
i =  1030
0.07677048
i =  1040
0.07662634
i =  1050
0.07659941
i =  1060
0.07645666
i =  1070
0.0763

0.07389889
i =  2250
0.073857315
i =  2260
0.07383844
i =  2270
0.07374621
i =  2280
0.073671594
i =  2290
0.0737102
i =  2300
0.073822945
i =  2310
0.07384045
i =  2320
0.073804095
i =  2330
0.07377014
i =  2340
0.073752366
i =  2350
0.07375009
i =  2360
0.073732086
i =  2370
0.07367855
i =  2380
0.07358818
i =  2390
0.07355283
i =  2400
0.07351915
i =  2410
0.07341411
i =  2420
0.07332576
i =  2430
0.07321298
i =  2440
0.0731905
i =  2450
0.073184006
i =  2460
0.073117554
i =  2470
0.07303183
i =  2480
0.072993286
i =  2490
0.07296964
i =  0
0.06296955
i =  10
0.06635919
i =  20
0.06352523
i =  30
0.06345855
i =  40
0.061902642
i =  50
0.058268502
i =  60
0.059435427
i =  70
0.06604523
i =  80
0.06847448
i =  90
0.06838631
i =  100
0.06864758
i =  110
0.069100544
i =  120
0.069612056
i =  130
0.07093799
i =  140
0.07139958
i =  150
0.072531395
i =  160
0.07357974
i =  170
0.07451532
i =  180
0.07482833
i =  190
0.075546786
i =  200
0.075881414
i =  210
0.07489341
i =  220
0.077411406

0.07450592
i =  1430
0.07478004
i =  1440
0.07486675
i =  1450
0.07477141
i =  1460
0.07476975
i =  1470
0.07470338
i =  1480
0.07473786
i =  1490
0.07479588
i =  1500
0.074735664
i =  1510
0.074691154
i =  1520
0.074637204
i =  1530
0.07459738
i =  1540
0.07474722
i =  1550
0.07487732
i =  1560
0.07490723
i =  1570
0.07484827
i =  1580
0.074705064
i =  1590
0.0745649
i =  1600
0.07447299
i =  1610
0.07441297
i =  1620
0.074364714
i =  1630
0.074253626
i =  1640
0.07422053
i =  1650
0.074174844
i =  1660
0.0741373
i =  1670
0.07410364
i =  1680
0.07398686
i =  1690
0.07386034
i =  1700
0.07370994
i =  1710
0.073598266
i =  1720
0.0735597
i =  1730
0.07366171
i =  1740
0.07374603
i =  1750
0.07374442
i =  1760
0.07374295
i =  1770
0.073728375
i =  1780
0.07375258
i =  1790
0.07387025
i =  1800
0.074058324
i =  1810
0.07415506
i =  1820
0.074079245
i =  1830
0.07395714
i =  1840
0.073910244
i =  1850
0.073860385
i =  1860
0.07374735
i =  1870
0.073611155
i =  1880
0.07350805
i =  1890
0.

0.07725597
i =  590
0.077625036
i =  600
0.077525795
i =  610
0.078017704
i =  620
0.078213155
i =  630
0.07794475
i =  640
0.07802794
i =  650
0.07815945
i =  660
0.07855676
i =  670
0.07860962
i =  680
0.07846961
i =  690
0.07837551
i =  700
0.07851729
i =  710
0.07855295
i =  720
0.07850085
i =  730
0.07824291
i =  740
0.07816641
i =  750
0.07812876
i =  760
0.0780462
i =  770
0.07830914
i =  780
0.07839098
i =  790
0.07827379
i =  800
0.07842374
i =  810
0.078390054
i =  820
0.078184985
i =  830
0.077960186
i =  840
0.07767504
i =  850
0.07732597
i =  860
0.07715227
i =  870
0.076948665
i =  880
0.07689774
i =  890
0.07674421
i =  900
0.07648367
i =  910
0.07622854
i =  920
0.07596694
i =  930
0.07597268
i =  940
0.07614581
i =  950
0.0760713
i =  960
0.07628316
i =  970
0.07629904
i =  980
0.0760728
i =  990
0.07598755
i =  1000
0.07607653
i =  1010
0.07649804
i =  1020
0.07684339
i =  1030
0.07675124
i =  1040
0.07660771
i =  1050
0.07658893
i =  1060
0.07647161
i =  1070
0.07638

0.07388519
i =  2220
0.07385048
i =  2230
0.07387495
i =  2240
0.07384623
i =  2250
0.07379712
i =  2260
0.07378174
i =  2270
0.073688105
i =  2280
0.07361228
i =  2290
0.07365424
i =  2300
0.07376262
i =  2310
0.073762745
i =  2320
0.07373493
i =  2330
0.073695004
i =  2340
0.07368133
i =  2350
0.073689155
i =  2360
0.07367019
i =  2370
0.07361084
i =  2380
0.073511355
i =  2390
0.07348123
i =  2400
0.07343753
i =  2410
0.07333815
i =  2420
0.07324499
i =  2430
0.073137924
i =  2440
0.073112756
i =  2450
0.07310432
i =  2460
0.07303269
i =  2470
0.07294632
i =  2480
0.07291125
i =  2490
0.0728898


In [610]:
data = Variable(torch.from_numpy(npdata[1:11]).type('torch.FloatTensor'))
# vk,pre_h1,pre_hk = rbm(data) 
vk,pre_h1,pre_hk = rbm.forward(data)
a = vk.t().max(1, keepdim=True)[1].data.t()
print(data)
print(a)
# print(a[data[1]>0])
# print(len(a[a>0]))

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  3.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])
tensor([[  9,  29,  27,  27,  29,  29,  29,  29,  27,  29]])


In [603]:
len(data[data>0])

355

tensor([ 3122.7827,  3227.4312,  3388.8630,  3192.9348,  3122.7827,
         3140.7627,  3120.8247,  3169.8062,  3567.2185,  3176.2527])